In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import clone
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import dataframe_image as dfi

import warnings
    
# warnings -> to silence warnings

warnings.filterwarnings("ignore")
np.set_printoptions(precision=5, suppress=True)


RANDOM_STATE = 1
N_JOBS = -1

class_names = ["Canis", "Dysg. Equisimilis", "Dysg. Dysgalactiae"]

map_target = {
    "Streptococcus canis": 0,
    "Streptococcus dysgalactiae subsp. equisimilis": 1,
    "Streptococcus dysgalactiae subsp. dysgalactiae": 2
}

map_target_inv = {
    0: "Strept. canis",
    1: "Strept. dysg. equisimilis",
    2: "Strept. dysg. dysgalactiae"
}
map_target_antibiotici = {
    "S" : 1,
    "NS" : 0
}

start = 9
n_antibiotici = 9
n_geni = 27
n_virulenza = 18
n_picchi = ['46','306']

In [13]:
for n in n_picchi:
    print('DATAFRAME CON '+n+' PICCHI')
    df = pd.read_csv("../data/Dati_Matemaldomics_"+n+"picchi.csv",
                    delimiter=';', index_col='ID Strain')
    n = int(n)
    modelli = {}
    df.rename(columns = {"Putative Subspecies":'subspecies'}, inplace = True)
    subspecies = df[['subspecies']]

    st = df[[df.columns[4]]]
    antibiotici = df[df.columns[start+n:start+n+n_antibiotici]]
    geni_antibiotici = df[df.columns[start+n+n_antibiotici:start+n+n_antibiotici+n_geni]]
    virulenza = df[df.columns[start+n+n_antibiotici+n_geni:start+n+n_antibiotici+n_geni+n_virulenza]]
    
    targets = {'antibiotici' : antibiotici,
                'geni_antibiotici' : geni_antibiotici,
                'virulenza' : virulenza}
    
    for str_target,target in targets.items():
        columns = target.columns
        for column in columns:
            if str_target == 'antibiotici':
                target[column] = df[column].map(map_target_antibiotici)
            rapporto = (target[column] == 0).sum() / target.shape[0]
            #if (antibiotici[column] == 0).all() or (antibiotici[column] == 1).all():
            print(column+" : "+str(rapporto))
            if rapporto < 0.15 or rapporto > 0.85:
                target.drop([column], axis=1, inplace=True)
        
        display(target)
        
    targets['subspecies'] = subspecies
    targets['st'] = st
    

DATAFRAME CON 46 PICCHI
Eritromicina : 0.461038961038961
Ceftiofur : 0.0
Tetraciclina : 0.5194805194805194
Gentamicina : 0.6233766233766234
Penicillina : 0.0
Ampicillina : 0.0
Sulfametossazolo_trimethoprim : 0.01948051948051948
Clindamicina : 0.2662337662337662
Enrofloxacin : 0.6688311688311688


,Eritromicina,Tetraciclina,Gentamicina,Clindamicina,Enrofloxacin
ID Strain,,,,,
V13,0,0,0,1,0
V142,0,1,1,1,0
V151,1,1,0,1,0
V160,1,0,0,1,0
V161,1,1,0,1,0
...,...,...,...,...,...
V800,1,0,1,1,0
V82,1,1,0,1,1
V90,1,0,0,1,0


aad(6) : 0.935064935064935
ANT(6)-Ia : 0.8246753246753247
APH(2'')-IIIa : 1.0
APH(3')-IIIa : 0.9025974025974026
catS : 0.9675324675324676
dfrF : 0.9805194805194806
E. faecalis chloramphenicol acetyltransferase : 0.9935064935064936
Erm(47) : 0.987012987012987
ErmB : 0.8181818181818182
fexA : 0.9935064935064936
L._reuteri cat-TC : 1.0
lmrP : 0.006493506493506494
lnuC : 0.987012987012987
lnuD : 0.9935064935064936
lsaC : 0.961038961038961
lsaE : 0.7857142857142857
mefE : 0.8506493506493507
optrA : 0.9935064935064936
poxtA : 0.9935064935064936
SAT-4 : 0.922077922077922
tet(40) : 0.987012987012987
tet(L) : 0.9935064935064936
tetM : 0.8181818181818182
tetO : 0.7402597402597403
tetS : 0.9805194805194806
tetT : 0.974025974025974
vatE : 0.9935064935064936


,ANT(6)-Ia,ErmB,lsaE,tetM,tetO
ID Strain,,,,,
V13,0,0,0,0,0
V142,0,0,0,0,0
V151,0,0,0,0,0
V160,0,0,0,0,0
V161,0,0,0,0,0
...,...,...,...,...,...
V800,0,0,0,1,0
V82,0,0,0,0,0
V90,0,0,0,0,0


fbp54 : 0.0
gbs0630 : 0.9935064935064936
gbs0631 : 0.9935064935064936
gbs0632 : 0.9935064935064936
hasC : 0.0
lmb : 0.9935064935064936
mf2 : 0.961038961038961
mf3 : 0.6753246753246753
scpA : 0.9935064935064936
sda : 0.8766233766233766
ska : 0.9935064935064936
slo : 0.9935064935064936
smeZ : 0.9935064935064936
spec : 0.974025974025974
speg : 0.9090909090909091
spek : 0.961038961038961
spel : 0.974025974025974
spem : 0.948051948051948


...

,Eritromicina,Tetraciclina,Gentamicina,Clindamicina,Enrofloxacin
ID Strain,,,,,
V13,0,0,0,1,0
V142,0,1,1,1,0
V151,1,1,0,1,0
V160,1,0,0,1,0
V161,1,1,0,1,0
...,...,...,...,...,...
V800,1,0,1,1,0
V82,1,1,0,1,1
V90,1,0,0,1,0


aad(6) : 0.935064935064935
ANT(6)-Ia : 0.8246753246753247
APH(2'')-IIIa : 1.0
APH(3')-IIIa : 0.9025974025974026
catS : 0.9675324675324676
dfrF : 0.9805194805194806
E. faecalis chloramphenicol acetyltransferase : 0.9935064935064936
Erm(47) : 0.987012987012987
ErmB : 0.8181818181818182
fexA : 0.9935064935064936
L._reuteri cat-TC : 1.0
lmrP : 0.006493506493506494
lnuC : 0.987012987012987
lnuD : 0.9935064935064936
lsaC : 0.961038961038961
lsaE : 0.7857142857142857
mefE : 0.8506493506493507
optrA : 0.9935064935064936
poxtA : 0.9935064935064936
SAT-4 : 0.922077922077922
tet(40) : 0.987012987012987
tet(L) : 0.9935064935064936
tetM : 0.8181818181818182
tetO : 0.7402597402597403
tetS : 0.9805194805194806
tetT : 0.974025974025974
vatE : 0.9935064935064936


,ANT(6)-Ia,ErmB,lsaE,tetM,tetO
ID Strain,,,,,
V13,0,0,0,0,0
V142,0,0,0,0,0
V151,0,0,0,0,0
V160,0,0,0,0,0
V161,0,0,0,0,0
...,...,...,...,...,...
V800,0,0,0,1,0
V82,0,0,0,0,0
V90,0,0,0,0,0


fbp54 : 0.0
gbs0630 : 0.9935064935064936
gbs0631 : 0.9935064935064936
gbs0632 : 0.9935064935064936
hasC : 0.0
lmb : 0.9935064935064936
mf2 : 0.961038961038961
mf3 : 0.6753246753246753
scpA : 0.9935064935064936
sda : 0.8766233766233766
ska : 0.9935064935064936
slo : 0.9935064935064936
smeZ : 0.9935064935064936
spec : 0.974025974025974
speg : 0.9090909090909091
spek : 0.961038961038961
spel : 0.974025974025974
spem : 0.948051948051948


,mf3
ID Strain,
V13,0
V142,1
V151,0
V160,0
V161,1
...,...
V800,0
V82,1
V90,0


In [14]:
for name_t,target in targets.items():
    for name in target:
        print(name)
        score_target = pd.read_csv('..\Risultati\Results_Def_'+str(n)+'picchi\Results_'+name+'_'+str(n)+'.csv')
        display(score_target)
        score_sort = score_target.sort_values(by=['Accuracy CV'], ascending=False)
        display(score_sort)
        #score_sort.to_csv('Risultati\Results_Def_'+str(n)+'picchi\Results_'+column+'_'+str(n)+'.csv', index = False)

Eritromicina


,Target,Model,Accuracy CV,St. Dev. CV,Precision CV,Recall CV,F1-Score CV,Accuracy
0,Eritromicina,LogisticRegression,0.512000,0.009798,0.262240,0.512000,0.346807,0.645161
1,Eritromicina,LogisticRegression_Best,0.568000,0.062738,0.571720,0.568000,0.563947,0.709677
2,Eritromicina,LogisticRegression_PCA,0.512000,0.009798,0.262240,0.512000,0.346807,0.645161
3,Eritromicina,LogisticRegression_Best_PCA,0.559667,0.099132,0.560408,0.559667,0.558473,0.645161
4,Eritromicina,Ridge,0.512000,0.009798,0.262240,0.512000,0.346807,0.645161
5,Eritromicina,Ridge_Best,0.551333,0.099367,0.555135,0.551333,0.531760,0.709677
6,Eritromicina,Ridge_PCA,0.512000,0.009798,0.262240,0.512000,0.346807,0.645161
7,Eritromicina,Ridge_Best_PCA,0.512000,0.051536,0.379727,0.512000,0.413185,0.580645
8,Eritromicina,DecisionTree,0.584333,0.065993,0.596378,0.584333,0.574232,0.645161
9,Eritromicina,DecisionTree_Best,0.561000,0.028550,0.579622,0.561000,0.534501,0.580645


,Target,Model,Accuracy CV,St. Dev. CV,Precision CV,Recall CV,F1-Score CV,Accuracy
13,Eritromicina,K-nn_Best,0.608333,0.100399,0.622926,0.608333,0.593680,0.645161
14,Eritromicina,K-nn_PCA,0.601000,0.088177,0.602817,0.601000,0.598540,0.645161
20,Eritromicina,BernoulliNB,0.600667,0.123848,0.603569,0.600667,0.585726,0.677419
21,Eritromicina,BernoulliNB_Best,0.592667,0.111378,0.595056,0.592667,0.577726,0.677419
12,Eritromicina,K-nn,0.584667,0.058485,0.584935,0.584667,0.580943,0.709677
8,Eritromicina,DecisionTree,0.584333,0.065993,0.596378,0.584333,0.574232,0.645161
11,Eritromicina,DecisionTree_Best_PCA,0.569000,0.020374,0.574277,0.569000,0.562943,0.516129
1,Eritromicina,LogisticRegression_Best,0.568000,0.062738,0.571720,0.568000,0.563947,0.709677
19,Eritromicina,RandomForest_Best_PCA,0.561333,0.043249,0.570386,0.561333,0.547403,0.580645
9,Eritromicina,DecisionTree_Best,0.561000,0.028550,0.579622,0.561000,0.534501,0.580645


FileNotFoundError: [WinError 2] Impossibile trovare il file specificato